In [1]:
import pandas as pd
import pandas_gbq

from github import Github
from google.cloud import bigquery
%load_ext google.cloud.bigquery

In [2]:
import pydata_google_auth
credentials = pydata_google_auth.get_user_credentials(
    ['https://www.googleapis.com/auth/bigquery'],
)

In [3]:
year = 'year.2017'
query = """SELECT created_at, repo.name FROM githubarchive.{0} WHERE RAND() < 1500/(SELECT COUNT(*) FROM `githubarchive.{0}`) ORDER BY created_at;""".format(year) 
project_id = 'cs515-term'

In [4]:
import pandas as pd
df = pd.read_gbq(query, project_id=project_id, dialect='standard', credentials=credentials)

repos = df['name']
dates = df['created_at']

Downloading: 100%|██████████| 1504/1504 [00:00<00:00, 7552.33rows/s]
